In [1]:
import betfairlightweight
import pandas as pd 
import numpy as np
from joblib import Parallel, delayed
import os
import matplotlib.pyplot as plt 

In [2]:
import datetime as dt

In [3]:
my_username = "##############"
my_password = "#################"
my_app_key = "################"

trading = betfairlightweight.APIClient(my_username, my_password, app_key=my_app_key, certs='/Users/mohamedbenmosbah/Documents/certs')

trading.login()

<LoginResource>

In [4]:
event_types = trading.betting.list_event_types()

In [5]:
trading.scores

In [6]:
sport = "Soccer"
plan = "Basic Plan"
collection_options = trading.historic.get_collection_options(
    sport, plan, 1, 3, 2021, 31, 3, 2021
)

In [7]:
# Print market types
market_types = [[*v.values()][0] for v in  collection_options['marketTypesCollection']]
[m for m in market_types if 'ODDS' in m]

['MATCH_ODDS',
 'MATCH_ODDS_AND_BTTS',
 'MATCH_ODDS_AND_OU_25',
 'FIRST_GOAL_ODDS',
 'CORNER_ODDS',
 'MATCH_ODDS_UNMANAGED',
 'BOOKING_ODDS',
 'MATCH_ODDS_AND_OU_35']

In [8]:
# Loads of files to load 
# Maybe use this to increase the limit but let's try to avoid that
trading.historic.read_timeout = 100

In [11]:
file_list = trading.historic.get_file_list(
    sport,
    plan,
    from_day=1,
    from_month=1,
    from_year=2023,
    to_day=31,
    to_month=1,
    to_year=2023,
    market_types_collection=["MATCH_ODDS"],
    countries_collection=["GB", "FR"],
    #file_type_collection=["M"],
)
print(len(file_list))

1229


In [30]:
os.chdir('/Users/mohamedbenmosbah/PythonFC/data/raw/')
def convert_json_to_df_and_save(file,output_folder='/Users/mohamedbenmosbah/PythonFC/data/'):
    try:
        json_file_name = trading.historic.download_file(file_path=file)
        json_file = pd.read_json(json_file_name,compression='bz2',lines=True)
        first_line = json_file[['mc']].iloc[0][0][0]
        first_time = json_file[['pt']].iloc[0][0]
        market_definitions = first_line['marketDefinition']
        regulator = market_definitions['regulators'][0]
        name_of_the_game = market_definitions['eventName']
        open_date = market_definitions['openDate']
        players_defition = market_definitions['runners']
        
        players = {p['id']:p['name'] for p in players_defition}
        df = pd.DataFrame(columns = [*players.keys()]+['Status','InPlay'])
        #print(players)
        for line_number in range(len(json_file)):
                time_stamp = json_file.iloc[line_number]['pt']
                market_changes = json_file.iloc[line_number]['mc'][0]
                if 'rc' in market_changes.keys():
                    # Check if there are Runner Changes
                    runner_changes = market_changes['rc']
                    for rc in runner_changes:
                        id_ = rc['id']
                        price = rc['ltp']
                        df.loc[time_stamp,id_] = price
                else:
                    _
                    #print('No Market Changes at line {}'.format(line_number))
                if 'marketDefinition' in market_changes.keys():
                    # Check if there are market definition changes (Status/InPlay)
                    market_defintion_changes = market_changes['marketDefinition']
                    inplay = market_defintion_changes['inPlay']
                    status = market_defintion_changes['status']
                    df.loc[time_stamp,'Status']= status
                    df.loc[time_stamp,'InPlay']= inplay
                else:
                    _
                    #print('No Market Defitions Changes at line {}'.format(line_number))
        last_line = json_file[['mc']].iloc[-1][0][0]
        last_time = json_file[['pt']].iloc[0][0]
        market_definitions = last_line['marketDefinition']
        open_date_end = market_definitions['openDate']
        players_defition = market_definitions['runners']
        players_result = {p['id']:p['status'] for p in players_defition}
        df['NameOfTheGame'] = name_of_the_game
        df['InPlay'].fillna(method='ffill',inplace=True)
        df['Status'].fillna(method='ffill',inplace=True)
        df['Status'].fillna('OPEN',inplace=True)
        df['InPlay'].fillna(False,inplace=True)

        df['Date'] = open_date_end
        df['Date'] = pd.to_datetime(df['Date'])
        df.reset_index(inplace = True)
        df.rename(columns={'index':'TimeStamp'},inplace=True)

        first_datetime = dt.datetime.strptime(open_date,'%Y-%m-%dT%H:%M:%S.%f%z')
        df['TimeFromFirstTime'] = df['TimeStamp']-first_time

        df['DateTime'] = df['TimeFromFirstTime'].apply(lambda x : first_datetime+dt.timedelta(seconds= x/1e3))
        df['Regulator'] = regulator
        df.rename(columns=players,inplace=True)
        
        # FillNa with complementary odds
        df.to_parquet(output_folder+name_of_the_game.replace(' ','_')+'_'+open_date)
        #return df
        return True
    except:
        return False

In [26]:
output_folder='/Users/mohamedbenmosbah/PythonFC/data/'
convert_json_to_df_and_save(file_list[142])

{2757228: 'Newington YC', 4918030: 'Knockbreda', 58805: 'The Draw'}


In [29]:
result = Parallel(n_jobs=-1,verbose=10)(delayed(convert_json_to_df_and_save)(file) for file in file_list)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1789s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1275s.) Setting batch_size=4.
[Parallel(n_jo

In [35]:
n = 0 
success = 0
for file in file_list:
    n += 1
    success = convert_json_to_df_and_save(file)
    if success: 
        success +=1 
    else : 
        print(file)
        break

{48350: 'Blackburn', 79343: 'Cardiff', 58805: 'The Draw'}
{7190809: 'Banbury Utd', 2145773: 'Brackley Town', 58805: 'The Draw'}
{1000084: 'Slough Town', 3396707: 'Oxford City', 58805: 'The Draw'}
{21943103: 'Farnborough FC', 4317868: 'Hungerford', 58805: 'The Draw'}
{1282862: 'Farsley Celtic', 598919: 'Bradford PA', 58805: 'The Draw'}
{473963: 'Angers', 44794: 'Lorient', 58805: 'The Draw'}
{10774: 'Nantes', 44800: 'Auxerre', 58805: 'The Draw'}
{305969: 'Toulouse', 8720753: 'AC Ajaccio', 58805: 'The Draw'}
{48224: 'Tottenham', 63908: 'Aston Villa', 58805: 'The Draw'}
{361706: 'Monaco', 669348: 'Brest', 58805: 'The Draw'}
{2662209: 'Gloucester', 1282865: 'Leamington', 58805: 'The Draw'}
{148733: 'Plymouth', 893582: 'MK Dons', 58805: 'The Draw'}
{56036: 'Millwall', 18565: 'Rotherham', 58805: 'The Draw'}
{2439369: 'Hampton and Richmond', 1034116: 'Worthing', 58805: 'The Draw'}
{3491521: 'Dartford', 3018410: 'Ebbsfleet Utd', 58805: 'The Draw'}
{4231709: 'Chorley', 4317866: 'AFC Fylde', 5880

{57423: 'Livingston', 10707: 'Motherwell', 58805: 'The Draw'}
{57425: 'St Johnstone', 48818: 'Dundee Utd', 58805: 'The Draw'}
{44790: 'Lille', 298233: 'Reims', 58805: 'The Draw'}
{78864: 'Brentford', 56323: 'Liverpool', 58805: 'The Draw'}
{44797: 'Montpellier', 44798: 'Marseille', 58805: 'The Draw'}
{44799: 'Rennes', 189829: 'Nice', 58805: 'The Draw'}
{62684: 'QPR', 48470: 'Sheff Utd', 58805: 'The Draw'}
{16821526: 'CS Korba', 21257783: 'ES Jerba Midoun', 58805: 'The Draw'}
{14188309: 'Club Olympique de Medenine', 9282745: 'AS Gabes', 58805: 'The Draw'}
{56343: 'Everton', 18567: 'Brighton', 58805: 'The Draw'}
{8089085: 'Penybont FC', 10119643: 'Airbus Uk Broughton', 58805: 'The Draw'}
{3664465: 'Caernarfon Town', 3667304: 'The New Saints', 58805: 'The Draw'}
{10227904: 'Pontypridd Town', 4099627: 'Bala Town', 58805: 'The Draw'}
{1096: 'Arsenal', 25422: 'Newcastle', 58805: 'The Draw'}
{48461: 'Leicester', 56764: 'Fulham', 58805: 'The Draw'}
{2757228: 'Newington YC', 4918030: 'Knockbreda

{47991: 'Crewe', 2662218: 'AFC Wimbledon', 58805: 'The Draw'}
{1282854: 'Blyth Spartans', 2145880: 'Kings Lynn', 58805: 'The Draw'}
{328049: 'Falkirk', 674746: 'Montrose', 58805: 'The Draw'}
{69717: 'Leyton Orient', 49205: 'Doncaster', 58805: 'The Draw'}
{219596: 'Chesterfield', 1703: 'West Brom', 58805: 'The Draw'}
{2145810: 'Chelmsford', 1282852: 'Chippenham', 58805: 'The Draw'}
{5114005: 'Havant and W', 1282860: 'Braintree', 58805: 'The Draw'}
{13959120: 'Dorking Wanderers', 69819: 'Oldham', 58805: 'The Draw'}
{256198: 'Harrogate Town', 148734: 'Colchester', 58805: 'The Draw'}
{256400: 'Cambridge Utd', 69821: 'Bristol Rovers', 58805: 'The Draw'}
{729806: 'Stenhousemuir', 122903: 'Stranraer', 58805: 'The Draw'}
{674748: 'St Mirren', 47993: 'Hearts', 58805: 'The Draw'}
{256226: 'York City', 6555176: 'Maidstone Utd', 58805: 'The Draw'}
{48349: 'Charlton', 148737: 'Lincoln', 58805: 'The Draw'}
{539982: 'Crusaders', 3611094: 'Dergview', 58805: 'The Draw'}
{54004: 'Tranmere', 1034133: 'Su

{14211737: 'H & W Welders', 7064523: 'Ballinamallard United', 58805: 'The Draw'}
{339725: 'Aberystwyth', 8089085: 'Penybont FC', 58805: 'The Draw'}
{25422: 'Newcastle', 48461: 'Leicester', 58805: 'The Draw'}
{48351: 'Man Utd', 48349: 'Charlton', 58805: 'The Draw'}
{8491983: 'Felixstowe & Walton Utd', 1433860: 'Bury Town', 58805: 'The Draw'}
{11750868: 'AFC Dunstable', 5212052: 'Cirencester Town', 58805: 'The Draw'}
{46955: 'Glenavon', 539982: 'Crusaders', 58805: 'The Draw'}
{539986: 'Newry', 166863: 'Coleraine', 58805: 'The Draw'}
{21511357: 'Huddersfield (Res)', 22034565: 'Fleetwood Town (Res)', 58805: 'The Draw'}
{10774: 'Nantes', 55271: 'Lyon', 58805: 'The Draw'}
{44800: 'Auxerre', 305969: 'Toulouse', 58805: 'The Draw'}
{669348: 'Brest', 44790: 'Lille', 58805: 'The Draw'}
{8720753: 'AC Ajaccio', 298233: 'Reims', 58805: 'The Draw'}
{309686: 'Clermont', 44799: 'Rennes', 58805: 'The Draw'}
{5419644: 'Sunderland (W)', 22340477: 'Leicester City (W)', 58805: 'The Draw'}
{761012: 'Charlton

{2662204: 'Buxton', 183604: 'Telford', 58805: 'The Draw'}
{48470: 'Sheff Utd', 69720: 'Stoke', 58805: 'The Draw'}
{48349: 'Charlton', 1117: 'Barnsley', 58805: 'The Draw'}
{256383: 'Rochdale', 148734: 'Colchester', 58805: 'The Draw'}
{1222695: 'Welling Utd', 7744599: 'Dulwich Hamlet', 58805: 'The Draw'}
{56299: 'Burnley', 48043: 'Coventry', 58805: 'The Draw'}
{3491521: 'Dartford', 2133279: 'Hemel Hempstead', 58805: 'The Draw'}
{2662209: 'Gloucester', 10915971: 'Hereford FC', 58805: 'The Draw'}
{223467: 'Luton', 1703: 'West Brom', 58805: 'The Draw'}
{13965046: 'AS Furiani Agliani', 11708140: 'US Lusitanos Saint Maur', 58805: 'The Draw'}
{18568: 'Southport', 4317866: 'AFC Fylde', 58805: 'The Draw'}
{103122: 'Reading', 62684: 'QPR', 58805: 'The Draw'}
{13959120: 'Dorking Wanderers', 18560: 'Kidderminster', 58805: 'The Draw'}
{1222691: 'Eastleigh', 1282860: 'Braintree', 58805: 'The Draw'}
{13965002: 'RC Grasse', 18197563: 'Aubagne FC', 58805: 'The Draw'}
{893582: 'MK Dons', 148737: 'Lincoln

{14846429: 'Peterborough Sports FC', 2662209: 'Gloucester', 58805: 'The Draw'}
{18568: 'Southport', 256306: 'Boston Utd', 58805: 'The Draw'}
{13133: 'Dundee', 48812: 'Dunfermline', 58805: 'The Draw'}
{62683: 'Swansea', 69745: 'Bristol City', 58805: 'The Draw'}
{183604: 'Telford', 18556: 'Kettering', 58805: 'The Draw'}
{36033: 'Forest Green', 48759: 'Birmingham', 58805: 'The Draw'}
{62505: 'Altrincham', 728405: 'Maidenhead', 58805: 'The Draw'}
{48044: 'Wolves', 56323: 'Liverpool', 58805: 'The Draw'}
{4317866: 'AFC Fylde', 2662204: 'Buxton', 58805: 'The Draw'}
{3018410: 'Ebbsfleet Utd', 2145810: 'Chelmsford', 58805: 'The Draw'}
{9792268: 'Atherton Collieries AFC', 1338704: 'Warrington Town', 58805: 'The Draw'}
{50808: 'Torquay', 8700096: 'Taunton Town', 58805: 'The Draw'}
{69746: 'Wigan', 223467: 'Luton', 58805: 'The Draw'}
{1703: 'West Brom', 219596: 'Chesterfield', 58805: 'The Draw'}
{7853878: 'Lincoln United FC', 5800583: 'Stocksbridge Park Steels', 58805: 'The Draw'}
{2437395: 'Solih

{7536171: 'Spennymoor Town', 1282865: 'Leamington', 58805: 'The Draw'}
{1282852: 'Chippenham', 1034116: 'Worthing', 58805: 'The Draw'}
{3491521: 'Dartford', 8700096: 'Taunton Town', 58805: 'The Draw'}
{2119313: 'Dover', 5114005: 'Havant and W', 58805: 'The Draw'}
{21943103: 'Farnborough FC', 3018410: 'Ebbsfleet Utd', 58805: 'The Draw'}
{2439369: 'Hampton and Richmond', 1034135: 'Tonbridge Angels', 58805: 'The Draw'}
{256465: 'St Albans', 1034124: 'Cheshunt', 58805: 'The Draw'}
{1222695: 'Welling Utd', 2133279: 'Hemel Hempstead', 58805: 'The Draw'}
{17815725: 'Chambery St O', 55271: 'Lyon', 58805: 'The Draw'}
{10598461: 'Cwmbran Celtic', 16961596: 'Penydarren FC', 58805: 'The Draw'}
{11386623: 'Annan', 12549240: 'Bonnyrigg', 58805: 'The Draw'}
{203715: 'Alloa', 328049: 'Falkirk', 58805: 'The Draw'}
{306045: 'Arbroath', 10707: 'Motherwell', 58805: 'The Draw'}
{48818: 'Dundee Utd', 6684879: 'Stirling University', 58805: 'The Draw'}
{539984: 'Larne', 539982: 'Crusaders', 58805: 'The Draw'}

{2224923: 'Thaon', 10774: 'Nantes', 58805: 'The Draw'}
{16630573: 'Le Puy', 20788794: 'Vierzon', 58805: 'The Draw'}
{9549626: 'Brighton (W)', 802851: 'Arsenal (W)', 58805: 'The Draw'}
{669348: 'Brest', 44793: 'Lens', 58805: 'The Draw'}
{46277118: 'Elgin City FC', 26138814: 'Lochee United', 58805: 'The Draw'}
{6586874: 'Reading U21', 6808104: 'Charlton U21', 58805: 'The Draw'}
{44792: 'Sedan', 269791: 'Le Mans', 58805: 'The Draw'}
{6601846: 'West Brom U21', 6731074: 'Leeds United U21', 58805: 'The Draw'}
{6615322: 'Tottenham U21', 6667755: 'Wolves U21', 58805: 'The Draw'}
{6692744: 'Derby U21', 6728422: 'Southampton U21', 58805: 'The Draw'}
{7840178: 'Hull City U21', 6692741: 'Nottingham Forest U21', 58805: 'The Draw'}
{51836521: 'Pays de Cassel', 489720: 'Paris St-G', 58805: 'The Draw'}
{10296975: 'Coleshill Town', 11662188: 'Corby Town FC', 58805: 'The Draw'}
{41814844: 'Darvel', 1189: 'Aberdeen', 58805: 'The Draw'}
{4231684: 'Aveley', 13557619: 'Hastings United', 58805: 'The Draw'}
{

{4099627: 'Bala Town', 339727: 'Connahs Quay', 58805: 'The Draw'}
{11753944: 'Bordeaux (W)', 9339203: 'Metz (W)', 58805: 'The Draw'}
{66265: 'Deveronvale', 47319202: 'Brora Rangers FC', 58805: 'The Draw'}
{20223749: 'Racing Colombes', 10621833: 'SM Caen II', 58805: 'The Draw'}
{309689: 'St Etienne', 44796: 'Sochaux', 58805: 'The Draw'}
{3491606: 'Haguenau', 21996491: 'Olympic Saint-Quentin', 58805: 'The Draw'}
{11386623: 'Annan', 674743: 'East Fife', 58805: 'The Draw'}
{729811: 'Stirling', 122903: 'Stranraer', 58805: 'The Draw'}
{7394884: 'Airdrieonians', 203715: 'Alloa', 58805: 'The Draw'}
{674742: 'Ross Co', 56086: 'Kilmarnock', 58805: 'The Draw'}
{18559: 'Cheltenham', 256373: 'Port Vale', 58805: 'The Draw'}
{674747: 'Raith', 56285: 'Inverness CT', 58805: 'The Draw'}
{180586: 'Portadown', 44330: 'Linfield', 58805: 'The Draw'}
{26334104: 'Rylands FC', 35783362: 'Liversedge', 58805: 'The Draw'}
{10369928: 'Belfort', 2740568: 'Epinal', 58805: 'The Draw'}
{539983: 'Institute', 539987: 'A

{1128445: 'Everton (W)', 5470466: 'Birmingham (W)', 58805: 'The Draw'}
{24306998: 'Partick Thistle (W)', 10161201: 'Hibernian (W)', 58805: 'The Draw'}
{57423: 'Livingston', 47993: 'Hearts', 58805: 'The Draw'}
{10770966: 'SC Amiens U19', 10632863: 'Le Havre AC U19', 58805: 'The Draw'}
{14616043: 'JA Drancy U19', 10722822: 'Stade Rennais U19', 58805: 'The Draw'}
{18567: 'Brighton', 56323: 'Liverpool', 58805: 'The Draw'}
{42319733: 'RC Lens (W)', 42383002: 'Thonon EG Geneve (W)', 58805: 'The Draw'}
{10722833: 'Valenciennes FC U19', 17149621: 'ES Troyes AC U19', 58805: 'The Draw'}
{669348: 'Brest', 473963: 'Angers', 58805: 'The Draw'}
{309686: 'Clermont', 10774: 'Nantes', 58805: 'The Draw'}
{44800: 'Auxerre', 44797: 'Montpellier', 58805: 'The Draw'}
{39047377: 'Wolverhampton Wanderers (W)', 22333102: 'West Ham United (W)', 58805: 'The Draw'}
{39041439: 'Ipswich (W)', 22340478: 'Lewes (W)', 58805: 'The Draw'}
{26389658: 'Burnley (W)', 22337938: 'Cardiff City FC (W)', 58805: 'The Draw'}
{514

In [36]:
success/n

0.0016273393002441008

In [38]:
len(os.listdir('/Users/mohamedbenmosbah/PythonFC/data/'))

1230